In [ ]:
import requests

## Suche

Der einfachste Weg, den gewünschten API-Aufruf zu konstruieren, führt über die Web-Oberfläche. Hier können die zunächst Suchparameter eingestellt werden, dann per Rechtsklick mit gedrückter Strg-Taste das Kontextmenü aufrufen und unter *Export* das gewünschte Format wählen. Ein Klick öffnet das Suchergebnis im gewünschten Format.

Die Webfrontend-Ansicht, die unter der URL <https://berlin.museum-digital.de/objects?s=place:10651> aufrufbar ist, entspricht <https://berlin.museum-digital.de/json/objects?&s=place%3A10651>.

In [ ]:
requests.get("https://berlin.museum-digital.de/json/objects?s=place:10651").json()

## Mehr als 24 Objekte auf einen Streich

Standardmäßig gibt das Webinterface und die API 24 Objekte je Aufruf aus. Dieser Wert kann auf max. 100 erhöht werden und mit dem Parameter `gbreitenat` übergeben werden. Über die Treffer kann mit dem Parameter `startwert` iteriert werden, indem `startwert` bei jedem Aufruf um `gbreitenat` erhöht wird. Um sich also eine größere Menge (sprich i.d.R. mehr als 100) an Objekten ausgeben zu lassen, bietet sich folgendes Vorgehen an: https://nat.museum-digital.de/objects?instnr=22&section=results_list&mode=grid&gbreitenat=100

In [ ]:
apiURL = "https://nat.museum-digital.de/json/objects?&s=gabel"

In [ ]:
objektIDs = list()

rows = 100
offset = 0
while True:   
    res = requests.get(apiURL + f"&startwert={offset}&gbreitenat={rows}")
    offset += rows
    if 'status' in res.json() and res.json()['status'] == 'Error':
        break
    else:
        for _ in res.json():
            objektIDs.append(_['objekt_id'])

Mithilfe der gesammelten IDs kann nun ein Gesamtabzug der Daten erstellt werden.

In [ ]:
from tqdm import tqdm # Zeigt den Forschritt an
dump = []
for ID in tqdm(objektIDs):
    obj = requests.get(f"https://nat.museum-digital.de/json/object/{ID}").json()
    dump.append(obj)

In [ ]:
herstellungsdaten = []
for obj in dump:
    for event in obj['object_events']:
        if event['event_type'] == 1:
            if 'time' in event:
                time_end =  event['time']['time_end']
                try:
                    herstellungsdaten.append(int(time_end))
                except Exception as e:
                    pass

In [ ]:
from matplotlib import pyplot as plt

plt.hist(herstellungsdaten, bins = 50)
plt.show()

## Liste der Institutionen

In [ ]:
res = requests.get("https://nat.museum-digital.de/institutions?output=json")

In [ ]:
# Die Institutionen mit den 5 größten Online-Beständen

# nach größe Sortieren
instSortiert = sorted([_ for _ in res.json()], key = lambda x:x['institution_objects'], reverse = True)

for _ in instSortiert[:6]:
    print(f"{_['institution_id']}, {_['institution_name']} ({_['institution_objects']} Objekte)")

### Sammlungen

In [ ]:
res = requests.get("https://global.museum-digital.org/json/collections")

In [ ]:
collections = []
for inst in res.json():
    for coll in inst['collection']:
        collections.append(coll['collection_id'])
        if coll['collection_no_of_subcollections'] > 0:
            for subcoll in coll['collection_subcollections']:
                collections.append(subcoll['subcollection_id'])

In [ ]:
len(collections)